## Sherman-Morrison-Woodbury formula

The Sherman-Morrison-Woodbury (SMW) formula is given by:

$$
(A + UV^T)^{-1} = A^{-1} - A^{-1} U \left( I + V^T A^{-1} U \right)^{-1} V^T A^{-1}
$$

This formula can be derived from :
$$
B^{-1}  = A^{-1} - B^{-1} (B-A) A^{-1}
$$


- https://en.wikipedia.org/wiki/Sherman%E2%80%93Morrison_formula


In [1]:
import numpy as np

# Function to compute (A + UV^T)^-1 
def sherman_morrison_woodbury(A_inv, U, V):
    k = min(V.shape)
    term = np.eye(k) + V.T @ A_inv @ U
    term_inv = np.linalg.inv(term)
    
    # Apply the Sherman-Morrison-Woodbury formula
    B_inv = A_inv - A_inv @ U @ term_inv @ V.T @ A_inv
    return B_inv


## Example (k=1)

In [2]:
import numpy as np

A = np.array([[4, 1, 2],
              [1, 3, 0],
              [2, 0, 5]])

u = np.array([[1], [0], [2]])
v = np.array([[2], [1], [1]])
A_inv = np.linalg.inv(A)
B_inv = sherman_morrison_woodbury(A_inv, u, v)

print("\nA:")
print(A)
print("\nu:")
print(u)
print("\nv:")
print(v)
print("\ninv(A):")
print(A_inv)
print("\ninv(A+u@v.T)")
print(B_inv)



A:
[[4 1 2]
 [1 3 0]
 [2 0 5]]

u:
[[1]
 [0]
 [2]]

v:
[[2]
 [1]
 [1]]

inv(A):
[[ 0.34883721 -0.11627907 -0.13953488]
 [-0.11627907  0.37209302  0.04651163]
 [-0.13953488  0.04651163  0.25581395]]

inv(A+u@v.T)
[[ 0.328125 -0.125    -0.140625]
 [-0.109375  0.375     0.046875]
 [-0.25      0.        0.25    ]]


## Example: k=2

In [3]:

# Example matrices
n = 3  # Dimension of A
k = 2 # rank of U and V

# Random matrices for A, U, and V
A = np.random.rand(n, n) 
U = np.random.rand(n, k) 
V = np.random.rand(n, k) 

# Compute the inverse of A
A_inv = np.linalg.inv(A)

# Compute the inverse of the perturbed matrix (A + UV^T) using the formula
B_inv = sherman_morrison_woodbury(A_inv, U, V)

# Output the results
print("Inverse of A (A_inv):")
print(A_inv)
print("\nInverse of (A + UV^T) (B_inv) using Sherman-Morrison-Woodbury formula:")
print(B_inv)


Inverse of A (A_inv):
[[-3.42925535 -0.32216644  2.09961154]
 [ 2.6764203   1.58239696 -1.71888934]
 [ 0.51388664 -1.3177128   0.89543044]]

Inverse of (A + UV^T) (B_inv) using Sherman-Morrison-Woodbury formula:
[[-3.88274093  0.25487055  1.82570536]
 [ 0.52944557  4.43895039 -3.79917769]
 [ 2.44970076 -3.87833091  2.67691264]]


## Example: incremental update

$$
(A+UV^T)^{-1} = (A + u_1 v_1^T + ... + u_n v_n^T)^{-1}
$$



In [4]:
# Compute inv(A+UV)
import numpy as np

n = 4
A = np.random.rand(n, n)
U = np.random.rand(n, n)
V = np.random.rand(n, n)
C_inv = np.linalg.inv(A)
true_inv = np.linalg.inv(A+U@V)

# incremental update
for i in range(n):
    u = U[:, i].reshape(-1, 1) # column vector
    v = V[i, :].reshape(-1, 1) # column vector
    C_inv = sherman_morrison_woodbury(C_inv, u, v)
    print(f"\ni={i} update:")
    print(C_inv)

print("\ntrue inv(A+U@V):")
print(true_inv)

if np.allclose(true_inv, C_inv):
    print("\ninv(A+U@V) == inv(A+u1@v1.T+...+un@vn.T)")
    print("Incremental update of the inverse is correct")



i=0 update:
[[-1.99950276 -0.18451536  3.83789451 -0.52283997]
 [ 2.01673129 -0.58185299 -1.76366677  0.37328924]
 [ 0.30421509  0.54414157 -2.5412146   1.61232731]
 [-0.45998748  1.37606254  0.18023458 -1.25792331]]

i=1 update:
[[-2.65176686 -2.11862921  7.34344203 -1.36838186]
 [ 2.29400535  0.24032871 -3.25385672  0.73272468]
 [ 0.53033566  1.21464133 -3.75648369  1.90545154]
 [-0.63891514  0.84550067  1.14186869 -1.48987051]]

i=2 update:
[[-0.87629399 -1.68126134  3.39383894 -0.3469809 ]
 [ 1.17013918 -0.03652314 -0.75377611  0.08618257]
 [-0.31279508  1.00694552 -1.88090914  1.420412  ]
 [ 0.20689699  1.05385702 -0.73967072 -1.00328841]]

i=3 update:
[[-0.64849436 -1.41836909  2.97498884 -0.30040868]
 [ 0.98991442 -0.24451159 -0.42240087  0.04933674]
 [-0.55275832  0.73001584 -1.43969407  1.37135301]
 [ 0.18080115  1.02374111 -0.6916889  -1.00862354]]

true inv(A+U@V):
[[-0.64849436 -1.41836909  2.97498884 -0.30040868]
 [ 0.98991442 -0.24451159 -0.42240087  0.04933674]
 [-0.552